# SCC0270 - Redes Neurais e Aprendizado Profundo
### Aula 7 - Prática (Redes neurais convolutivas)
**Daniel Penna Chaves Bertazzo - 10349561**

# Exercício 1
A função de ativação *softmax* é dada pela fórmula
<br><br>
$$ S(y_i) = \frac{e^{y_i}} {\sum_{j=1}^{K} e^{y_j}} $$
<br>
onde $K$ é o número de classes presentes no *dataset*. Tal função de ativação possui como entrada um vetor de $K$ elementos ($\in \mathbb{R}$) e o normaliza para uma distribuição de probabilidade (outro vetor com $K$ elementos, onde cada elemento representa uma probabilidade proporcional aos exponenciais dos valores de entrada). No contexto de redes neurais, é normalmente utilizada na camada de saída, pois pega como entrada os valores gerados pelas ativações dos neurônios da penúltima camada e os transforma em um vetor onde cada $S(y_i)$ representa a probabilidade da classe $i$ ser a predição correta para aquela instância do *dataset*. Por exemplo:
<br><br>
Suponha que possuímos um *dataset* com 3 classes diferentes e uma rede neural treinada para prever corretamente à qual classe uma dada instância pertence. Ao fornecer à rede tal exemplo, suponha que ela retorne como resultado o vetor:
<br><br>
$$ \begin{bmatrix} {0.02 \\
                    0.98 \\
                    0.00} \end{bmatrix} $$
<br>
onde cada linha representa uma classe $y$. Nesse caso, temos que
* $P(y=1) = 2\%$
* $P(y=2) = 98\%$
* $P(y=3) = 0\%$
<br>
Logo, de acordo com o modelo, a instância em questão tem 98% de probabilidade de ser da classe 2, sendo classificada como tal.

# Exercício 2
A função de ativação ReLU é dada pela fórmula
<br>
$$ y = \max(0, x) $$
<br>
e possui comportamento constante em zero para todos os valores menores que zero e linear crescente para qualquer valor maior que zero. Essa função é a mais comumente utilizada em redes neurais convolutivas pois, além de ser rápida de calcular, ela não satura (não converge para um valor fixo de acordo com o crescimento de $y$, diferente da ativação sigmóide) e é ativada de forma esparsa, ou seja, a maioria das entradas $y$ serão transformadas em zero (não ativam o neurônio), simulando de forma mais realista a forma como o cérebro biológico, principal insipiração para redes neurais, funciona.

# Exercício 3 

In [1]:
import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, MaxPooling2D, Flatten
from tensorflow.keras.utils import to_categorical

In [2]:
# Carrega o dataset
(X_train, y_train), (X_test, y_test) = tf.keras.datasets.mnist.load_data()

In [3]:
# Ajusta o formato dos dados para serem compativeis com o modelo de rede neural
# X_train: formato antigo = (60000, 28, 28) --> formato novo: (60000, 28, 28, 1)
# X_test: formato antigo = (10000, 28, 28) --> formato novo: (10000, 28, 28, 1)

X_train = X_train.reshape(X_train.shape[0], X_train.shape[1], X_train.shape[2], 1)
X_test  = X_test.reshape(X_test.shape[0], X_test.shape[1], X_test.shape[2], 1)

In [4]:
# Ajusta o formato dos targets para serem compativeis com o modelo de rede neural
# Faz one-hot encoding

y_train = to_categorical(y_train)
y_test  = to_categorical(y_test)

### Modelo 1
* **Primeira camada:** convolução com 32 filtros 3x3 e ativação relu + max_pooling 2x2

* **Segunda camada:** convolução com 64 filtros 3x3 e ativação relu + max_pooling 2x2

* **Última camada:** *fully connected* com 10 neurônios (número de classes) e ativação softmax

In [5]:
model1 = Sequential()

# Primeira camada de convolucao e max-pooling
model1.add(Conv2D(filters=32, kernel_size=(3, 3), activation='relu', input_shape=X_train.shape[1:]))
model1.add(MaxPooling2D(pool_size=(2, 2)))

# Segunda camdada de convolicao e max-pooling
model1.add(Conv2D(filters=64, kernel_size=(3, 3), activation='relu'))
model1.add(MaxPooling2D(pool_size=(2, 2)))

# "Achata" os dados para um vetor unidimensional
model1.add(Flatten())

# Ultima camada: fully connected com 10 neuronios (1 para cada classe) e
# ativacao softmax para obter as probabilidades
model1.add(Dense(10, activation='softmax'))

In [6]:
model1.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 26, 26, 32)        320       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 13, 13, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 11, 11, 64)        18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 5, 5, 64)          0         
_________________________________________________________________
flatten (Flatten)            (None, 1600)              0         
_________________________________________________________________
dense (Dense)                (None, 10)                16010     
Total params: 34,826
Trainable params: 34,826
Non-trainable params: 0
____________________________________________________

In [7]:
# Compila o modelo
# adam optimizer: otimizador que ajusta o learning rate
# categorical_crossentropy: usada para problemas de classificacoes com multiplas classes
# accuracy: metrica usada para ver a eficacia do modelo
model1.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [8]:
# Treina o modelo
model1.fit(X_train, y_train, validation_split=0.3, epochs=3)

Train on 42000 samples, validate on 18000 samples
Epoch 1/3
42000/42000 [==============================] - 27s 631us/sample - loss: 0.3396 - accuracy: 0.9399 - val_loss: 0.0959 - val_accuracy: 0.9736
Epoch 2/3
42000/42000 [==============================] - 33s 793us/sample - loss: 0.0759 - accuracy: 0.9784 - val_loss: 0.0729 - val_accuracy: 0.9794
Epoch 3/3
42000/42000 [==============================] - 31s 728us/sample - loss: 0.0581 - accuracy: 0.9822 - val_loss: 0.0895 - val_accuracy: 0.9758


In [10]:
# Faz predicoes com conjunto de teste
y_pred1 = model1.predict(X_test)

In [12]:
m1 = tf.keras.metrics.CategoricalAccuracy()
m1.update_state(y_test, y_pred1)
print("Acuracia no conjunto de teste = ", m1.result().numpy())

Acuracia no conjunto de teste =  0.9782


### Modelo 2
* **Primeira camada:** convolução com 64 filtros 3x3 e ativação relu + max_pooling 2x2

* **Segunda camada:** convolução com 32 filtros 3x3 e ativação relu + max_pooling 2x2

* **Última camada:** *fully connected* com 10 neurônios (número de classes) e ativação softmax


In [13]:
model2 = Sequential()

# Primeira camada de convolucao e max-pooling
model2.add(Conv2D(filters=64, kernel_size=(3, 3), activation='relu', input_shape=X_train.shape[1:]))
model2.add(MaxPooling2D(pool_size=(2, 2)))

# Segunda camdada de convolicao e max-pooling
model2.add(Conv2D(filters=32, kernel_size=(3, 3), activation='relu'))
model2.add(MaxPooling2D(pool_size=(2, 2)))

# "Achata" os dados para um vetor unidimensional
model2.add(Flatten())

# Ultima camada: fully connected com 10 neuronios (1 para cada classe) e
# ativacao softmax para obter as probabilidades
model2.add(Dense(10, activation='softmax'))

In [14]:
model2.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_4 (Conv2D)            (None, 26, 26, 64)        640       
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 13, 13, 64)        0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 11, 11, 32)        18464     
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 5, 5, 32)          0         
_________________________________________________________________
flatten_2 (Flatten)          (None, 800)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 10)                8010      
Total params: 27,114
Trainable params: 27,114
Non-trainable params: 0
__________________________________________________

In [16]:
# Compila o modelo
# adam optimizer: otimizador que ajusta o learning rate
# categorical_crossentropy: usada para problemas de classificacoes com multiplas classes
# accuracy: metrica usada para ver a eficacia do modelo
model2.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [17]:
# Treina o modelo
model2.fit(X_train, y_train, validation_split=0.3, epochs=3)

Train on 42000 samples, validate on 18000 samples
Epoch 1/3
42000/42000 [==============================] - 33s 795us/sample - loss: 0.5202 - accuracy: 0.9117 - val_loss: 0.1310 - val_accuracy: 0.9612
Epoch 2/3
42000/42000 [==============================] - 36s 852us/sample - loss: 0.1060 - accuracy: 0.9677 - val_loss: 0.0977 - val_accuracy: 0.9691
Epoch 3/3
42000/42000 [==============================] - 35s 829us/sample - loss: 0.0785 - accuracy: 0.9761 - val_loss: 0.1014 - val_accuracy: 0.9691


In [18]:
# Faz predicoes com conjunto de teste
y_pred2 = model2.predict(X_test)

In [19]:
m2 = tf.keras.metrics.CategoricalAccuracy()
m2.update_state(y_test, y_pred2)
print("Acuracia no conjunto de teste = ", m2.result().numpy())

Acuracia no conjunto de teste =  0.9733


# Resultados e conclusões
<br><br>
![Tabela](tabela.png)
<br><br>
Percebe-se que o modelo *multilayer perceptron* utilizado no trabalho avaliativo possui uma acurácia maior durante a fase de treinamento, porém ambos os modelos convolutivos apresentam resultados melhores na fase de teste, ou seja, eles possuem uma maior capacidade de generalização do aprendizado.
<br><br>
Portanto, conclui-se que para este *dataset*, que é composto por imagens, um modelo convolutivo mostra-se mais apropriado.